# Clustering Crypto

In [106]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [107]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv').set_index('Unnamed: 0')
crypto_df.index.name = None
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [108]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading==True]
crypto_df.IsTrading.unique()

array([ True])

In [109]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns='IsTrading', inplace=True)
crypto_df.columns

Index(['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [110]:
# Remove rows that have at least 1 null value.
for column in crypto_df.columns:
    print(f'Column {column} has {crypto_df[column].isnull().sum()} null values.')

crypto_df.dropna(inplace=True)

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 459 null values.
Column TotalCoinSupply has 0 null values.


In [111]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df.TotalCoinsMined>0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [112]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame(crypto_df['CoinName'])
names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [113]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [114]:
# Use get_dummies() to create variables for text features.
names = pd.get_dummies(crypto_df[['Algorithm','ProofType']])
names_df = pd.DataFrame(names)
crypto_df.drop(['Algorithm','ProofType'], axis=1, inplace=True)
X=pd.concat([names_df, crypto_df], axis=1)
X


,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,TotalCoinsMined,TotalCoinSupply
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.199995e+01,42
404,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.055185e+09,532000000
1337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.927942e+10,314159265359
BTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.792718e+07,21000000
ETH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.076842e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.000000e+09,2000000000
GAP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.493105e+07,250000000
BDX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.802226e+08,1400222610
ZEN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7.296538e+06,21000000


In [115]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)
print(X)

[[-0.0433963  -0.0433963  -0.0433963  ... -0.0433963  -0.11710817
  -0.1528703 ]
 [-0.0433963  -0.0433963  -0.0433963  ... -0.0433963  -0.09396955
  -0.145009  ]
 [-0.0433963  -0.0433963  -0.0433963  ... -0.0433963   0.52494561
   4.48942416]
 ...
 [-0.0433963  -0.0433963  -0.0433963  ... -0.0433963  -0.09561336
  -0.13217937]
 [-0.0433963  -0.0433963  -0.0433963  ... -0.0433963  -0.11694817
  -0.15255998]
 [-0.0433963  -0.0433963  -0.0433963  ... -0.0433963  -0.11710536
  -0.15285552]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE

In [ ]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE


Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
# YOUR CODE HERE

# Fit the model
# YOUR CODE HERE

# Predict clusters
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE


In [ ]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
